# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
#print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [5]:
df = pd.read_csv('event_datafile_new.csv')
display(df.head())

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
try:
    session.set_keyspace('music')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### First Query

###### For table songs_by_userSession, the sessionId was used as a partition key because the queries will filter by this column. itemlnSession was used as clustering columns to help make up a unique key.


In [9]:
query = "CREATE TABLE IF NOT EXISTS songs_by_userSession "
query = query + "( \
sessionId int, \
itemlnSession int, \
artist text, \
length float, \
song text, \
Primary KEY (sessionId, itemlnSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [10]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_userSession ( \
        sessionId, \
        itemlnSession, \
        artist, \
        length, \
        song)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]), line[9]))
        except Exception as e:
            print(e)

###### 1. Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [12]:
artists=[]
songs=[]
lengths=[]
query = "select artist, song,length from songs_by_userSession WHERE sessionId=338 AND itemlnSession= 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    artists.append(row.artist)
    songs.append(row.song)
    lengths.append(row.length)

data = {'Artist': artists, 'Song': songs, "length": lengths}
df = pd.DataFrame(data)
df

,Artist,Song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### Second Query

#### For table songs_by_userId, the userId and sessionId were used as composite partition keys because the queries will filter by these columns. column_b and column_c were used as clustering columns to help make up a unique key.


In [13]:
query = "CREATE TABLE IF NOT EXISTS songs_by_userId "
query = query + "( \
userId int, \
sessionId int, \
itemlnSession int, \
artist text, \
firstName text, \
lastName text, \
song text, \
Primary KEY ((userId, sessionId), itemlnSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [14]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_userId ( \
        userId, \
        sessionId, \
        itemlnSession, \
        artist, \
        firstName, \
        lastName, \
        song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        try:
            session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[1], line[4], line[9]))
        except Exception as e:
            print(e)

###### 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [15]:
artists=[]
songs=[]
fnames=[]
lnames=[]
query = "select artist, song, firstName, lastName from songs_by_userId WHERE userId=10 AND sessionId= 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    artists.append(row.artist)
    songs.append(row.song)
    fnames.append(row.firstname)
    lnames.append(row.lastname)
    
data = {'Artist': artists, 'Song': songs, 'First Name':fnames, 'Last Name':lnames}
df = pd.DataFrame(data)
df

,Artist,Song,First Name,Last Name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### Third Query

#### For table username_by_song, song column was used as a partition key because the queries will filter by this column. firstName, lastName and userId different users may have the same first/last name but userIds are already unique identifiers) were used as clustering columns to help make up a unique key.


In [16]:
query = "CREATE TABLE IF NOT EXISTS username_by_song "
query = query + "( \
song text, \
firstName text, \
lastName text, \
userId int, \
Primary KEY (song, firstName, lastName, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO username_by_song ( \
        song, \
        firstName, \
        lastName, \
        userId)"
        query = query + "VALUES (%s, %s, %s, %s)"
        try:
            session.execute(query, (line[9], line[1], line[4], int(line[10])))
        except Exception as e:
            print(e)

###### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [18]:
fnames = []
lnames = []
query = "select firstName, lastName from username_by_song WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    fnames.append(row.firstname)
    lnames.append(row.lastname)
    
data = list(zip(fnames, lnames))
df = pd.DataFrame(data, columns=['First Name', 'Last Name'])
df

,First Name,Last Name
0,Jacqueline,Lynch
1,Sara,Johnson
2,Tegan,Levine


### Drop the tables before closing out the sessions

In [19]:
query = "DROP TABLE IF EXISTS music.songs_by_userSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query2 = "DROP TABLE IF EXISTS music.songs_by_userId"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

query3 = "DROP TABLE IF EXISTS music.username_by_song"
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()